In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import create_map
from pyspark.sql.functions import array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):") or "1"
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file: cos_sum.tri



********************
tin_directory:  


Is the data stored in hdfs(0) or SigSpatial(1): 0


tin_basename:  cos_sum.tri
tin_filename:  cos_sum
tin_extension:  .tri

********************


In [4]:
# filtra is the order of each vertex, the order is obtained by ranking elevation values of vertices
filtra = input("Do you have filtration data?") or "yes"
# filtra = 'yes'

if filtra.lower() == 'no':    
    Basic_Data = input("Do you have basic pts and tri data?")
    
graph = input("Is this graph VE (0) or ET (1)?") or "1"
Num_executor = input("spark.executor.instances:") or "4"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "28g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "16g"

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor

Num_core_per_driver = '5'
Memory_driver = '12g'
MemoryOverhead_driver = '4g'

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "200"

Do you have filtration data? yes
Is this graph VE (0) or ET (1)? 0
spark.executor.instances: 
spark.executor.cores: 
spark.executor.memory? Please end with 'g': 
spark.executor.memoryOverhead? Please end with 'g': 
spark.sql.shuffle.partitions: 


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

# set the Spark app name
spark_app_name = "Visualize_gradient_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '200') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: Visualize_gradient_cos_sum_04032024_1715


In [8]:
import sys
sys.path.append("/local/data/yuehui/pyspark/FormanGradient/code")

In [9]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

In [10]:
# read_ver_order() is a function used to read vertices from a csv file
def read_ver_order(filtra, directory, tin_filename):
    '''
    this function has three input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a TIN without extension, e.g., Canyon_Lake_Gorge_TX
    '''
    if filtra.lower() == 'yes': # 'yes' means we have already computed the filtra value for each vertex
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True), \
            StructField("self_order",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_filtra_pts_origin.csv'
    
        df_ver_order = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    if filtra.lower() == 'no': # 'no' means we need to rank vertices to get their filtra values
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'
    
        df_ver_origin = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        
        print("Number of partitions for df_ver_origin:", df_ver_origin.rdd.getNumPartitions())
        
        # define a window for the ordering
        # row_number() function along with partitionBy() of other column populates the row number by group
        # Since we want to order the whole DataFrame, so we don't need the partitionBy() function
        w = Window().orderBy(col('ele').asc())
        df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    return df_ver_order

In [11]:
# read vertices
df_ver_order = read_ver_order(filtra, directory, tin_filename)
df_ver_order.printSchema()

Number of partitions for df_ver_order: 1
root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)



In [12]:
# read_tri_order() is a function used to read triangles from a csv file
def read_tri_order(filtra, directory, tin_filename):    
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a tin without extension, e.g., 827_monviso
    '''
    if filtra.lower() == 'yes':
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_filtra_tri_origin.csv'
    else: # filtra.lower() == 'no'
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_tri_origin.csv'
        
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])

    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())
        
    return df_tri_origin

In [13]:
# read triangles
df_tri_origin = read_tri_order(filtra, directory, tin_filename)
df_tri_origin.printSchema()

Number of partitions for df_tri_origin: 1
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- tri_order: integer (nullable = true)



In [14]:
# replace_ver() replaces the original index of each vertex with filtra value in df_tri_origin
def replace_ver(df_ver_order, df_tri_origin):
    '''
    df_ver_order: a DataFrame storing sorted vertices with filtra values
    df_tri_origin: a DataFrame storing triangles from a TIN
    '''
    df_tri_order_v1 = df_tri_origin.join(df_ver_order, df_tri_origin.v1 == df_ver_order.self_index, "inner")
    df_tri_order_v1 = df_tri_order_v1.select(col("tri_order"), col("v1"), col("v2"), col("v3"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))

    df_tri_order_v2 = df_tri_order_v1.join(df_ver_order, df_tri_order_v1.v2 == df_ver_order.self_index, "inner")
    df_tri_order_v2 = df_tri_order_v2.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))

    df_tri_order_v3 = df_tri_order_v2.join(df_ver_order, df_tri_order_v2.v3 == df_ver_order.self_index, "inner")
    df_tri_order_v3 = df_tri_order_v3.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))

    df_tri_order = df_tri_order_v3.select(col("tri_order"), col("r1"), col("r2"), col("r3"), col("r1_ele"), col("r2_ele"), col("r3_ele"))

    return df_tri_order

In [15]:
# replaces original vertex index with filtra values
df_tri_order = replace_ver(df_ver_order, df_tri_origin)
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



In [16]:
# sort the extreme vertices of a triangle in ascending order, e.g., (2,5,3) -> (2,3,5)

'''
def get_tri_array(r1, r2, r3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    tri = [r1, r2, r3]    
    tri.sort(reverse=True)
    
    return tri

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_tri_array_udf = udf(get_tri_array, ArrayType(IntegerType()))
df_tri_order = df_tri_order.withColumn("tri", get_tri_array_udf(df_tri_order.r1, df_tri_order.r2, df_tri_order.r3))
'''

df_tri_order = df_tri_order.withColumn("tri_origin", F.array("r1", "r2", "r3")).withColumn("tri_ele_origin", F.array("r1_ele", "r2_ele", "r3_ele"))
df_tri_order = df_tri_order.withColumn("tri", sort_array("tri_origin", False)).drop("tri_origin") # sort_array("tri", False), False means descending order
df_tri_order = df_tri_order.withColumn("tri_ele", sort_array("tri_ele_origin", False)).drop("tri_ele_origin") # sort_array("tri", False), False means descending order

df_tri_order.printSchema()
df_tri_order.rdd.getNumPartitions()
#df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



200

### get VT relation

In [17]:
# union and group vertices and get the preliminary VT relation
def grp_union(df_tri_order):
    '''
    df_tri_order: a DataFrame storing sorted extreme vertices of each triangle
    '''
    # groupby and collect_set
    df_tri_group1 = df_tri_order.groupBy('r1','r1_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group2 = df_tri_order.groupBy('r2','r2_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group3 = df_tri_order.groupBy('r3', 'r3_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    
    union12 = df_tri_group1.union(df_tri_group2) # the title of union12 will be the same as df_sj_group1
    union123 = union12.union(df_tri_group3) # the title of union123 will be the same as union12, so as df_sj_group1

    union123_group = union123.groupBy('r1','r1_ele').agg(collect_list('multi_tri').alias('multi_tri_list'), collect_list('multi_tri_ele').alias('multi_tri_ele_list'))
    
    # print("the schema of union123_group:", union123_group.printSchema())
    return union123_group

In [18]:
union123_group = grp_union(df_tri_order)
union123_group.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_tri_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: float (containsNull = true)



In [19]:
# sort the triangles in the preliminary VT relation
def get_multi_tri_order(tri_list, tri_ele_list):
    '''
    get_multi_tri_order is used to obtain the adjacent triangles, the results are in ascending order, but the triangle itself is in descending
    e.g., [[3, 2, 1], [5, 3, 2], [5, 3, 1]]
    tri_list: an array of array of array of integer
    tri_ele_list: an array of array of array of float
    '''

    tri = []
    tri_ele = []
    # pt_list.append(pt_self) # if we don't calculate roughness, we don't need self vertex
    for i in range(len(tri_list)):
        for j in range(len(tri_list[i])):
            # if tri_list[i][j] not in tri: # we do not need to check if it exists in tri_list, because we can prove that each one is unique
            tri.append(tri_list[i][j]) # update() will add multiple elements to a set, while add() only add one element to a set   
            tri_ele.append(tri_ele_list[i][j])
    
    tri_copy = copy.deepcopy(tri) # deep copy tri
    tri.sort() # sort the list of array, e.g., tri=[[3, 2, 1], [5, 2, 1], [5, 3, 1]], after sorting, [[3, 2, 1], [5, 2, 1], [5, 3, 1]]
    
    tri_ele_sort = []
    # sort tri_ele according to the same rule as tri
    for i in range(len(tri)):
        index_in_tri_origin = tri_copy.index(tri[i])
        tri_ele_sort.append(tri_ele[index_in_tri_origin])
    
    return tri, tri_ele_sort

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html

# StructType for get_multi_tri_order        
get_multi_tri_order_schema = StructType([
    StructField("multi_tri_sort", ArrayType(ArrayType(IntegerType())),True), 
    StructField('multi_tri_ele_sort',ArrayType(ArrayType(FloatType())),True)
])

get_multi_tri_order_udf = udf(get_multi_tri_order, get_multi_tri_order_schema)

df_VT = union123_group.withColumn("multi_tri_order", get_multi_tri_order_udf(union123_group.multi_tri_list, union123_group.multi_tri_ele_list))
df_VT = df_VT.select(col("r1").alias("Ver"), col("multi_tri_order.multi_tri_sort").alias("VT_filtra"), col("multi_tri_order.multi_tri_ele_sort").alias("VT_ele"))
df_VT.printSchema()
print("number of partitions for df_VT:", df_VT.rdd.getNumPartitions())

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_VT: 200


### obtain the lower star of each vertex

In [20]:
# obtain Lower Star
def get_LS(Ver, VT_filtra, VT_ele):
# get_LS is used to obtain the edges and triangles of LS in ascending order
# VV: an array of VV relation in ascending order
    VV = set(num for sublist in VT_filtra for num in sublist if num != Ver)
    VV = sorted(VV)
        
    LS_edge = []
    for i in range(len(VV)):
        if VV[i] < int(Ver):
            LS_edge.append([int(Ver),VV[i]])
        
    LS_tri = []
    LS_tri_ele = []
    
    for i in range(len(VT_filtra)):
        if Ver >= max(VT_filtra[i]): # Ver_ele is not the maximum vertex
            LS_tri.append(VT_filtra[i])
            LS_tri_ele.append(VT_ele[i])
                
    return LS_edge, LS_tri, LS_tri_ele

# StructType for get_multi_tri_order        
get_LS_schema = StructType([
    StructField("LS_edge", ArrayType(ArrayType(IntegerType())),True), 
    StructField("LS_tri_filtra", ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_tri_ele',ArrayType(ArrayType(FloatType())),True)
])

get_LS_udf = udf(get_LS, get_LS_schema)

df_LS = df_VT.withColumn("LS", get_LS_udf(df_VT.Ver, df_VT.VT_filtra, df_VT.VT_ele))
df_LS = df_LS.select("Ver", "VT_filtra", col("LS.LS_edge").alias("LS_edge"), col("LS.LS_tri_filtra").alias("LS_tri_filtra"), col("LS.LS_tri_ele").alias("LS_tri_ele"))
df_LS.printSchema()
print("number of partitions for df_LS:", df_LS.rdd.getNumPartitions())
# df_LS_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_LS: 200


### compute Forman Gradient

In [21]:
from collections import deque

In [22]:
# get the Forman gradient from lower stars
def get_Forman(Ver, LS_edge, LS_filtra, LS_ele):
    '''
    get_Forman is used to compute Forman gradient
    LS_edge is triangles of lower star in ascending order
    LS_tri is triangles of lower star in ascending order
    '''
    # inner udf to get the number of unpaired edges for a triangle
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_ET_inner):
        # compute the number of unpaired edges
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = []
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[1])
            # if type(pair_vec[0]) != int and len(pair_vec[0]) == 2: # pair_vec[0] is a paired edge, like in (edge, tri)
                # paired_edge.append(pair_vec[0])
            # if len(pair_vec[1]) == 2: # pair_vec[1] is a paired edge, like in (ver, edge)
                # paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_ET_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[0])
                
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            
        return num_unpaired_edge
    
    # inner udf to get the edge which is paired with a triangles
    def pair_an_edge_from_PQzero(alpha_inner, PQzero_inner):
    # alpha_inner is a triangle in [r0, r1, r2] format, and the vertext's elevation is in descending order
    # PQzero_inner is a queue storing all other edges
        alpha_e0 = [alpha_inner[0], alpha_inner[1]]
        alpha_e1 = [alpha_inner[0], alpha_inner[2]]
        for edge_inner in PQzero_inner:
            if edge_inner == alpha_e0:
                return edge_inner
            if edge_inner == alpha_e1:
                return edge_inner
            
    crit_cell = [] # crit_cell will store critical simplices
    # Forman_vec_pair = [] # Forman_vec_pair will store all Forman gradient pairs as a vector
    Forman_vec_pair_VE = []
    Forman_vec_pair_ET = []
    if (len(LS_edge)+len(LS_filtra)) == 0: # ver is a local minimum
        crit_cell.append([Ver]) # store minimum as an array [Ver] though it has only one element
    elif len(LS_edge) > 0:
        Forman_vec_pair_VE.append([Ver, LS_edge[0]]) # LS_edge[0] is the minimum edge since it is in ascending order
        
        # define two queues, PQzero storing all other edges, PQone storing all triangles which have only one unpaired edge
        PQzero = deque()
        PQone = deque()
        PQone_ele = deque()
        for i in range(1, len(LS_edge)): # add the other edges to PQzero
            PQzero.append(LS_edge[i])
            
        cell_temp_index_of_remove = []
        index_temp = 0
        for cell_temp in LS_filtra:
            if num_unpaired_edges(cell_temp, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                PQone.append(cell_temp) # storing cells with one unpaired edges, which are triangles in TIN
                PQone_ele.append(LS_ele[index_temp])
                # remove cell_temp from LS_filtra
                # LS_filtra.remove(cell_temp)
                cell_temp_index_of_remove.append(index_temp)
            index_temp += 1
            
        # remove cell_temp from LS_filtra and LS_ele
        for i in reversed(cell_temp_index_of_remove):
            del LS_filtra[i]
            del LS_ele[i]
                
        # sort PQone in ascending order, the initial queue is already sorted when creating
        # PQone = sorted(PQone) # sort() is not supported but we can use sorted()
        # PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
        while len(PQone) > 0 or len(PQzero) > 0:
            while len(PQone) > 0:
                # alpha = PQone.popleft() # alpha is a triangle in the PQone
                # alpha should be a triangle with lower elevation when PQone have more than one triangle
                if len(PQone) > 1 and abs(PQone_ele[0][1]-PQone_ele[1][1]) < 0.000001 and PQone_ele[0][2] > PQone_ele[1][2]:
                    alpha = PQone[1] # alpha is the second triangle
                    del PQone[1]
                    del PQone_ele[1]
                else:
                    alpha = PQone.popleft() # alpha is the first triangle, which is the same as alpha=PQone[0], then del PQone[0]
                    del PQone_ele[0]
                
                if num_unpaired_edges(alpha, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 0:
                    PQzero.append(alpha)
                else:
                    pair_alpha = pair_an_edge_from_PQzero(alpha, PQzero) # pair_alpha is an edge, [pair_alpha, alpha] will be a new paired vector
                    # if pair_alpha in PQzero:
                    PQzero.remove(pair_alpha) # remove pair_alpha from PQzero
                    # Forman_vec_pair.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    Forman_vec_pair_ET.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    
                    beta_cell_temp_index_of_remove = []
                    beta_index_temp = 0
                    for cell_beta in LS_filtra: # this can be optimized
                        if cell_beta not in PQone and num_unpaired_edges(cell_beta, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                            PQone.append(cell_beta)
                            PQone_ele.append(LS_ele[beta_index_temp])
                            # remove cell_beta from LS_tri
                            # LS_tri.remove(cell_beta)
                            beta_cell_temp_index_of_remove.append(beta_index_temp)
                        beta_index_temp += 1
                    
                    # remove cell_beta from LS_filtra and LS_ele
                    for i in reversed(beta_cell_temp_index_of_remove):
                        del LS_filtra[i]
                        del LS_ele[i]
                    
                    PQone = sorted(PQone)
                    PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
            if len(PQzero) > 0:
                gama = PQzero.popleft() # gama is a critical simplex
                crit_cell.append(gama)
                
                gama_cell_temp_index_of_remove = []
                gama_index_temp = 0
                for cell_gama in LS_filtra: # this can be optimized
                    if cell_gama not in PQone and num_unpaired_edges(cell_gama, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                        PQone.append(cell_gama)
                        PQone_ele.append(LS_ele[gama_index_temp])
                        # remove cell_gama from LS_tri
                        # LS_tri.remove(cell_gama)
                        gama_cell_temp_index_of_remove.append(gama_index_temp)
                    gama_index_temp += 1
                    
                # remove cell_beta from LS_filtra and LS_ele
                for i in reversed(gama_cell_temp_index_of_remove):
                    del LS_filtra[i]
                    del LS_ele[i]
                        
                PQone = sorted(PQone)
                PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
                
    return crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET

# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for ET pairs        
add_multi_ET_schema = StructType([
    StructField("ET_pair_E", ArrayType(IntegerType()),True), 
    StructField('ET_pair_T',ArrayType(IntegerType()),True)
])
 
# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("ET_pair", ArrayType(add_multi_ET_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [23]:
df_Forman = df_LS.withColumn("LS_Forman", get_Forman_udf(df_LS.Ver, df_LS.LS_edge, df_LS.LS_tri_filtra, df_LS.LS_tri_ele))
df_Forman = df_Forman.select(col("Ver"), col("VT_filtra"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- ET_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)

The number of par

### get Forman gradient

In [24]:
# file_df_Forman = directory + '/' + tin_filename + '_sortForman' + '.parquet'
# df_Forman = spark.read.parquet(file_df_Forman)
# df_Forman.printSchema()

### Extract critical vertices, edeges, and triangles

In [25]:
df_crit = df_Forman.select("Ver", col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 200
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [26]:
# split the Critical column to obtain the critical vertices, edges, and triangles
def get_crit_VET(crit_cell):
    if len(crit_cell) < 1: # crit_cell is empty, there are no critical simplices within this lower star
        return
    else:
        # initialize the critical vertices, critical edges, and critical triangles
        crit_ver = None
        crit_edge = []
        crit_tri = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1: # icrit_cell_temp stores a critical vertex
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2: # icrit_cell_temp stores critical edges
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3: # icrit_cell_temp stores critical triangles
                crit_tri.append(icrit_cell_temp)
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        return crit_ver, crit_edge, crit_tri
        
# the whole StructType
get_crit_VET_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VET_udf = udf(get_crit_VET, get_crit_VET_schema)

df_crit_VET = df_crit.withColumn("crit_cell", get_crit_VET_udf(df_crit.Critical))
print("number of partitions for df_crit_VET:", df_crit_VET.rdd.getNumPartitions())
df_crit_VET.printSchema()
# df_crit_VET.show()

number of partitions for df_crit_VET: 200
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [27]:
# to get critical vertices
df_crit_V = df_crit_VET.select(col("Ver"), col("crit_cell.crit_ver").alias("Min_ver")).na.drop()
df_crit_V.printSchema()
print("number of partitions for df_crit_V:", df_crit_V.rdd.getNumPartitions())
# df_crit_V.show()

root
 |-- Ver: integer (nullable = true)
 |-- Min_ver: integer (nullable = true)

number of partitions for df_crit_V: 200


In [28]:
# to get critical edges
df_crit_E_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_edge").alias("Saddle_edge_init"))

# there may be multiple saddles inside one lower star
df_crit_E = df_crit_E_init.select(col("Ver"), explode("Saddle_edge_init").alias("Saddle_edge"))
df_crit_E.printSchema()
print("number of partitions for df_crit_E:", df_crit_E.rdd.getNumPartitions())
# df_crit_E.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_E: 200


In [29]:
# to get critical triangles
df_crit_T_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tri_init").alias("Max_tri")).na.drop()
df_crit_T.printSchema()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
# df_crit_T.show()

root
 |-- Ver: integer (nullable = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_T: 200


##### collect the critical vertices, edges, and triangles as a global array

In [30]:
# critical vertices array
df_crit_V_col = df_crit_V.collect()

# critical edges array
df_crit_E_col = df_crit_E.collect()

# critical triangles array
df_crit_T_col = df_crit_T.collect()

In [31]:
df_crit_V_col

[Row(Ver=277, Min_ver=277),
 Row(Ver=1, Min_ver=1),
 Row(Ver=2, Min_ver=2),
 Row(Ver=0, Min_ver=0),
 Row(Ver=3, Min_ver=3)]

In [32]:
df_crit_E_col

[Row(Ver=322, Saddle_edge=[322, 274]),
 Row(Ver=299, Saddle_edge=[299, 272]),
 Row(Ver=321, Saddle_edge=[321, 273]),
 Row(Ver=345, Saddle_edge=[345, 275]),
 Row(Ver=278, Saddle_edge=[278, 277])]

In [33]:
df_crit_T_col

[Row(Ver=624, Max_tri=[624, 620, 607])]

### sort df_ver_order according to the self_order value, which is the filtration value

In [34]:
df_ver_order = df_ver_order.sort(col('self_order'), ascending=True)

# collect df_ver_order as a global array 
df_ver_order_col = df_ver_order.collect()

In [35]:
df_ver_order_col[0:5]

[Row(x=3.141590118408203, y=3.141590118408203, ele=-2.0, self_index=208, self_order=0),
 Row(x=9.424779891967773, y=3.141590118408203, ele=-2.0, self_index=224, self_order=1),
 Row(x=3.141590118408203, y=9.424779891967773, ele=-2.0, self_index=608, self_order=2),
 Row(x=9.424779891967773, y=9.424779891967773, ele=-2.0, self_index=624, self_order=3),
 Row(x=3.141590118408203, y=2.748889923095703, ele=-1.9238799810409546, self_index=183, self_order=4)]

In [112]:
df_ver_order_col[0]['x']

3.141590118408203

##### collect VE pairs

In [60]:
df_Forman_VE_pair_init = df_Forman.select(col("Ver"), col("Forman.VE_pair").alias("VE_pair"))
df_Forman_VE_pair_init.printSchema()
# df_Forman_VE_pair = df_Forman_VE_pair_init.select(explode(""))

root
 |-- Ver: integer (nullable = true)
 |-- VE_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [61]:
df_Forman_VE_pair_init = df_Forman_VE_pair_init.sort(col('Ver'), ascending=True)
df_Forman_VE_pair_init_col = df_Forman_VE_pair_init.collect()

In [63]:
df_Forman_VE_pair_init_col[0:5]

[Row(Ver=0, VE_pair=[]),
 Row(Ver=1, VE_pair=[]),
 Row(Ver=2, VE_pair=[]),
 Row(Ver=3, VE_pair=[]),
 Row(Ver=4, VE_pair=[Row(VE_pair_V=4, VE_pair_E=[4, 0])])]

In [68]:
df_Forman_VE_pair_init_col[4]['VE_pair']

[Row(VE_pair_V=4, VE_pair_E=[4, 0])]

In [71]:
df_Forman_VE_pair_init_col[4]['VE_pair'][0]['VE_pair_E']

[4, 0]

##### collect ET pairs

In [48]:
# Method: firstly extract df_Forman_ET_pair from df_Forman
df_Forman_ET_pair_init = df_Forman.select(col("Forman.ET_pair").alias("ET_pair"))
df_Forman_ET_pair_init.printSchema()
# df_Forman_ET_pair.show()

root
 |-- ET_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [49]:
df_Forman_ET_pair_init_explode = df_Forman_ET_pair_init.select(explode("ET_pair"))
df_Forman_ET_pair_init_explode.printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- ET_pair_E: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- ET_pair_T: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)



In [50]:
df_Forman_ET_pair = df_Forman_ET_pair_init_explode.select(df_Forman_ET_pair_init_explode.col.ET_pair_E.alias("ET_pair_E"), df_Forman_ET_pair_init_explode.col.ET_pair_T.alias("ET_pair_T"))
df_Forman_ET_pair.printSchema()

root
 |-- ET_pair_E: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- ET_pair_T: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [51]:
df_Forman_ET_pair_col = df_Forman_ET_pair.collect()

In [89]:
df_Forman_ET_pair_col[0]

Row(ET_pair_E=[250, 184], ET_pair_T=[250, 184, 172])

In [92]:
len(df_Forman_ET_pair_col[0]['ET_pair_E'])

2

### save Forman gradient

In [ ]:
# given the triangle id, obtain the xyz of the baricenter of this triangle
def baricenterXYZ(tri_id):
    tri_end_pts = df_graph_V2_node_col[tri_id]['tri']
    v_0 = tri_end_pts[0]
    v_1 = tri_end_pts[1]
    v_2 = tri_end_pts[2]
    x = (df_ver_order_col[v_0]['x'] + df_ver_order_col[v_1]['x'] + df_ver_order_col[v_2]['x'])/3.0
    y = (df_ver_order_col[v_0]['y'] + df_ver_order_col[v_1]['y'] + df_ver_order_col[v_2]['y'])/3.0
    z = (df_ver_order_col[v_0]['ele'] + df_ver_order_col[v_1]['ele'] + df_ver_order_col[v_2]['ele'])/3.0
    return x, y, z

In [ ]:
# given the triangle id, obtain the xyz of the baricenter of this triangle
def baricenterXYZ_from_ptArray(v_0, v_1, v_2):
    x = (df_ver_order_col[v_0]['x'] + df_ver_order_col[v_1]['x'] + df_ver_order_col[v_2]['x'])/3.0
    y = (df_ver_order_col[v_0]['y'] + df_ver_order_col[v_1]['y'] + df_ver_order_col[v_2]['y'])/3.0
    z = (df_ver_order_col[v_0]['ele'] + df_ver_order_col[v_1]['ele'] + df_ver_order_col[v_2]['ele'])/3.0
    return x, y, z

In [ ]:
# given the array of vertex id of an edge, obtain the xyz of its mid-point
def mid_pt_edge(v_0, v_1):
    x = (df_ver_order_col[v_0]['x'] + df_ver_order_col[v_1]['x'])/2.0
    y = (df_ver_order_col[v_0]['y'] + df_ver_order_col[v_1]['y'])/2.0
    z = (df_ver_order_col[v_0]['ele'] + df_ver_order_col[v_1]['ele'])/2.0
    return x, y, z

In [ ]:
# given teh triangle ids of two adjacent triangle, obtain the xyz of the mid-point of the common edge
def mid_pt_XYZ(tri_id_0, tri_id_1):
    tri_end_pts_0 = df_graph_V2_node_col[tri_id_0]['tri']
    tri_end_pts_1 = df_graph_V2_node_col[tri_id_1]['tri']
    # get the common edge of these two triangles
    intersection = set(tri_end_pts_0) & set(tri_end_pts_1)
    edge = list(intersection)
    v_0 = edge[0]
    v_1 = edge[1]
    x = (df_ver_order_col[v_0]['x'] + df_ver_order_col[v_1]['x'])/2.0
    y = (df_ver_order_col[v_0]['y'] + df_ver_order_col[v_1]['y'])/2.0
    z = (df_ver_order_col[v_0]['ele'] + df_ver_order_col[v_1]['ele'])/2.0
    return x, y, z

In [82]:
# get the gradient vertor of a VE pair
def gradient_VE(v_order, paired_e):
    v_x = df_ver_order_col[v_order]['x']
    v_y = df_ver_order_col[v_order]['y']
    v_z = df_ver_order_col[v_order]['ele']
            
    # get the other vertex different from v_order
    if v_order == paired_e[0]:
        v_other = paired_e[1]
    else:
        v_other = paired_e[0]
     
    # xyz of the other vertex
    v_other_x = df_ver_order_col[v_other]['x']
    v_other_y = df_ver_order_col[v_other]['y']
    v_other_z = df_ver_order_col[v_other]['ele']
    
    # the vector is the difference between mid-point and an end point
    vec_x = (v_other_x + v_x)/2.0 - v_x
    vec_y = (v_other_y + v_y)/2.0 - v_y
    vec_z = (v_other_z + v_z)/2.0 - v_z
    
    return [vec_x, vec_y, vec_z]

In [115]:
def write_gradient(output_vtk):
    vertices_num = len(df_Forman_VE_pair_init_col) + len(df_Forman_ET_pair_col)
        
    with open(output_vtk, 'w', newline='') as writer:
        writer.write('# vtk DataFile Version 2.0\n')
        writer.write('\n')
        writer.write('ASCII\n')
        writer.write('DATASET UNSTRUCTURED_GRID\n')
        writer.write('\n')
        writer.write(f'POINTS {vertices_num} float\n')
        
        # write vertices involved in VE pairs and critical vertices, they are mixed
        v_crit_or_not = []
        v_gradient_vect = []
        for i in range(len(df_Forman_VE_pair_init_col)):
            v_order = df_Forman_VE_pair_init_col[i]['Ver']
            v_x = df_ver_order_col[v_order]['x']
            v_y = df_ver_order_col[v_order]['y']
            v_z = df_ver_order_col[v_order]['ele']
            writer.write(f"{v_x:.6f} {v_y:.6f} {v_z:.6f}\n")
            
            # check if this is a critical vertex
            if len(df_Forman_VE_pair_init_col[i]['VE_pair']) == 0:
                # this is a critical vertex
                v_crit_or_not.append(0)
                # append its gradient vector as [0,0,0]
                v_gradient_vect.append([0,0,0])
            else:
                # this is a regular vertex
                v_crit_or_not.append(-1)
                
                # calculate the gradient of this vertex
                paired_e = df_Forman_VE_pair_init_col[i]['VE_pair'][0]['VE_pair_E']
                v_gradient_vect.append(gradient_VE(v_order, paired_e))
                
        
        # write edges involved in ET pairs, critical edges are not considered
        e_gradient_vect = []
        for i in range(len(df_Forman_ET_pair_col)):
            # write the mid-point of an edge
            e = df_Forman_ET_pair_col[i]['ET_pair_E']
            e_mid_x, e_mid_y, e_mid_z = mid_pt_edge(e[0], e[1])
            writer.write(f"{e_mid_x:.6f} {e_mid_y:.6f} {e_mid_z:.6f}\n")
            
            # calculate the gradient of this edge
            paired_t = df_Forman_ET_pair_col[i]['ET_pair_T']
            paired_t_bariCen_x, paired_t_bariCen_y, paired_t_bariCen_z = baricenterXYZ_from_ptArray(paired_t[0], paired_t[1], paired_t[2])
            e_gradient_vect.append([paired_t_bariCen_x - e_mid_x, paired_t_bariCen_y - e_mid_y, paired_t_bariCen_z - e_mid_z])
            
        
        # write triangles involved in the ET pairs, critical triangles are not considered
        writer.write('\n')
        writer.write('\n')
        tri_num = len(df_Forman_ET_pair_col)
        writer.write(f"CELLS {tri_num} {tri_num*4}\n")
        for i in range(len(df_Forman_ET_pair_col)):
            paired_t = df_Forman_ET_pair_col[i]['ET_pair_T']
            writer.write(f"3 {paired_t[0]} {paired_t[1]}  {paired_t[2]}\n")
            
        writer.write('\n')
        writer.write(f"CELL_TYPES {tri_num}\n")
        for i in range(tri_num):
            writer.write('5 ')
            
        writer.write('\n')
        writer.write('\n')
        
        writer.write(f'POINT_DATA {vertices_num}\n')
        writer.write(f'VECTORS vector float\n')
        for i in range(len(df_Forman_VE_pair_init_col)):
            writer.write(f'{v_gradient_vect[i][0]:.6f} {v_gradient_vect[i][1]:.6f} {v_gradient_vect[i][2]:.6f} ')
            
        for i in range(len(df_Forman_ET_pair_col)):
            writer.write(f'{e_gradient_vect[i][0]:.6f} {e_gradient_vect[i][1]:.6f} {e_gradient_vect[i][2]:.6f} ')
            
        writer.write('\n')
        writer.write('\n')
        
        writer.write(f'FIELD FieldData 1\n')
        writer.write(f'critical 1 {vertices_num} int\n')
        for i in range(len(df_Forman_VE_pair_init_col)):
            writer.write(f'{v_crit_or_not[i]} ')
            
        for i in range(len(df_Forman_ET_pair_col)):
            writer.write(f'-1 ')
        
        writer.write('\n\n')
        writer.close()
        
    print("Writing gradient complete.")

In [116]:
output_vtk_grad = '/home/qiany/yuehui/pyspark/data/' + tin_filename + '_gradient_' + date_name + '.vtk'

write_gradient(output_vtk_grad)

Writing gradient complete.
